In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from io import StringIO
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
table_content = soup.find('table').text

In [6]:
table_list = table_content.split('\n')

table_list = list(filter(None, table_list))
print(table_list[:20])
labels = ['PostalCode', 'Borough', 'Neighborhood']
records = table_list[3:]

['Postcode', 'Borough', 'Neighbourhood', 'M1A', 'Not assigned', 'Not assigned', 'M2A', 'Not assigned', 'Not assigned', 'M3A', 'North York', 'Parkwoods', 'M4A', 'North York', 'Victoria Village', 'M5A', 'Downtown Toronto', 'Harbourfront', 'M5A', 'Downtown Toronto']


In [7]:
column1 = []
for i in range(0,len(records)):
        if i%3 == 0:
            column1.append(records[i])

column2 = []
for i in range(0,len(records)):
        if (i-1)%3 == 0:
            column2.append(records[i])

column3 = []
for i in range(0,len(records)):
        if (i-2)%3 == 0:
            column3.append(records[i])

records_arranged = list(zip(column1, column2, column3))
records_arranged[:3]

[('M1A', 'Not assigned', 'Not assigned'),
 ('M2A', 'Not assigned', 'Not assigned'),
 ('M3A', 'North York', 'Parkwoods')]

In [8]:
df = pd.DataFrame.from_records(records_arranged, columns=labels)

#drop rows with Borough="Not assigned"
df = df[df["Borough"] != "Not assigned"]

df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [9]:
customjoin = lambda a: ", ".join(a) 

#create a custom function to join the Borough values with comma, seperate them by comma and only return the first (since they are equal)
#I simply wanted to leave the Borough value unchanged. I'm sure there is a more elegant way, but this one works fine.
onereturn= lambda a: ",".join(a).rsplit(",")[0]

#group dataframe by PostalCode and join entries 
df_final = df.groupby('PostalCode').agg({'Borough': onereturn, 'Neighborhood': customjoin})

#check if the join works as expected:
df_final.head(4)

,Borough,Neighborhood
PostalCode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn


In [10]:
df_final=df_final.reindex(columns=['Borough', 'Neighborhood'])
df_final = df_final.reset_index()
df_final.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
